# Projeto 01: Reconhecimento de Dígitos

### João Victor Soares Silva | Mat: 20210027300
### Yhasmim de Souza Tigre | Mat: 20210026966 

## Pré-Processamento

In [2]:
def reduzir (nome : str, arq : str, lista_intensidades : list, lista_simetrias : list) -> None:

    '''
    Função que gera o arquivo .csv com os novos parâmetros de intensidade e simetria.
    Parâmetros: nome (string) - nome do .csv que será criado;
    arq (string) -  nome do .csv que será recebido para pegar as labels;
    lista_intensidades (list) - lista que contém a intensidade dos pixels de cada dado do dataset;
    lista_simetrias (list) - lista que contém a simetria de cada dado do dataset.

    Retorno: redux (csv) - um .csv com dimensões reduzidas para diminuir a complexidade dos dados,
    constando apenas com os parâmetros label, intensidade e simetria.  
    '''

    redux = open (f'{nome}.csv', 'w')
    aux = open (arq, 'r')

    redux.write ('label, intensidade, simetria\n')

    i = 0
    for linha in aux:
        redux.write (f'{linha[0]}, {lista_intensidades[i]}, {lista_simetrias[i]}\n')
        i += 1

    redux.close()
    aux.close()

In [ ]:
def intensidade (arq : str) -> list:
    lista_intensidades = []
    aux = open (arq, 'r')

    valor = ''
    for linha in aux:
        soma = 0

        for i in range (2, len(linha)): #Pulando o label
            if (linha[i] != ';'):
                valor += linha[i]
            
            else:
                soma += int(valor)
                valor = ''
        
        lista_intensidades.append((soma + int(valor))/255)
        valor = ''
    
    aux.close()

    return lista_intensidades

In [ ]:
def simetria (arq : str) -> list:
    lista_simetrias = []
    aux = open (arq, 'r')

    #Simetria vertical:
    for linha in aux:
        for i in range (2, len(linha)):
            continue

    #Simetria horizontal:

    aux.close()

    return lista_simetrias

## Algoritmos

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

## Classificação dos Dígitos 1x5

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Classificação de Dígitos Completo

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Comparação dos Classificadores

## Implementações Avançadas

### Weight-Decay para a Regressão Logística

### Definindo a Melhor Ordem